# STEP 3: Data Cleansing and Creation of Final Data File

## Polisticians Semester Project

In [1]:
import pandas as pd
import os
import numpy as np
import math

## We need to create a function to be used on the DF to further clean up our speakers

In [2]:
def speaker_map(row): # function to find if a transcript row contains a speaker

    if type(row['Transcript']) is str: # if a transcript line is a string
        
        # there are two main instances of whether a transcript line contains
        # a speaker: if there's a colon in the line, or if there's a semi-colon
        # There are some other instances, but we clean those up later via 
        # a mapping file 
        
        if row['Transcript'].find(':') > 0: # find returns -1 if ':' isn't found
            f = row['Transcript'].find(':') # return the index of the colon
        else: 
            f = row['Transcript'].find(';') # same as above except w/ semi-colon
        if f > 0:            
            return row['Transcript'][0:f] # if either ';' or ':' were found, return the name
        else:
            return np.nan # if neither found, return nan

# there are some lines in the transcripts that have colons when a speaker is making a point (i.e. not a name). We want to 
# flag where the speaker_map returns a name, not one of these instances. We use this later to pull out the actual 
# transcript words and not just the name

def find_colon(row): 
    i = 0
    try:
        for r in row['Transcript'].split(' '): 
            i += 1
            if r.find(':') > 0:
                
                # return the position of the colon if it's found in one of the
                # first 4 words
                if i < 4:
                    return i 
                else: # return 0 if the colon shows up later
                    return 0
        # If we get here, there isn't a colon later in the word        
        return 0
    except:
        return 0


## Now that we've created these functions, we can continue with the data-cleansing process

In [3]:
# functions are now defined. Will use those later in the apply function 

df = pd.read_csv('Transcripts_df.csv') # read the transcript data

#print(df.columns) # print column names

# apply the speaker_map function to isolate the speaker
df['Speaker_Clean'] = df.apply(speaker_map, axis = 1)

# Need to remove formality from the names
l = ['MR. ', 'MRS.', 'MS.']

# Replace the formal greetings
for i in l:
    df['Speaker_Clean'] = df['Speaker_Clean'].str.replace(i, '')

# We want to forward fill the speaker column if blank
df['Speaker_Clean'] = df['Speaker_Clean'].fillna(method='ffill')

# Some steps to cleanse the speaker name even further. If the speaker contains more than three words, we logically
# conclude that it isn't actually a speaker, so we set the Speaker_fin column to be the Speaker brought in from the 
# webscraper

df['Speaker_fin'] = np.where(df['Speaker_Clean'].str.split(' ').str.len() > 3, df['Speaker'], df['Speaker_Clean'])
df['Speaker_fin'] = np.where(df['Speaker_fin'].isnull(), df['Speaker_Clean'], df['Speaker_fin']) # fill nulls w clean spkr

df['word_count'] = df['Transcript'].str.split(' ').str.len() # get total word count, including the speaker
df['flag'] = df.apply(find_colon, axis = 1) # apply the find_colon function

df['word_count'] = df['word_count'] - df['flag'] # true wordcount is the length of the transcript line minus the flag
df = df.drop(['flag', 'Speaker'], axis=1) # cleanup DF by dropping some columns
df.fillna('xx') # fill the remaining NAs in the DF with xx

#print(df.head())
#print(df.dtypes)

pd.Series(df['Speaker_fin'].unique()).to_csv('speakers.csv') # used this file to test for abnormal speakers



## We've now performed data cleansing on the speakers from the transcript files. We sent the unique elements of the column to a csv. We analyzed those data for any abnormalities as part of our testing and used it as the basis of a mapping file.
## Additionally, we created a mapping file containing more data, like party, number of electoral college votes, etc.
## This file also cleaned up any abnormalities in the data, such as when a candidate ran for VP and P in different years, and when two candidates have the same last name, like the Bushes or Clintons. We need to specify the year in these instances since the Speaker_fin alone isn't enough.

In [4]:
speaker_map = pd.read_csv('speaker_map.csv') # read the initial mapping file to clean up the last names
clean_map = pd.read_csv('map_file_final.csv') # read in additional data

#print(speaker_map.head())
#print(speaker_map.dtypes)

# this mapping file cleans up any transcripts with abnormalities, like typos or when a transcript was in the speaker
# field rather than a name
df = df.join(speaker_map.set_index('Speaker_fin'), rsuffix='_map', on = 'Speaker_fin') 

## We also need to bring in the 2020 debates and append to the rest of the data

In [5]:
# read in 2020 debates, which don't need the same level of cleansing as the CPD debate transcripts
df2 = pd.read_csv('2020debates.csv')
df2['word_count'] = df2['Transcript'].str.split(' ').str.len() # compute the word count by spaces

df = df.append(df2) # append. All of the transcripts are now in df

C:\Users\fuent\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


## Now that we have a full set of data, the only thing left is to join our additional fields and final speaker identification

In [6]:
#print(df.columns)

# the 2020 debates need slight manipulation to turn the Debate column into a date, so we just nest some replace methods
df['Debate_date'] = df['Debate'].str.replace('Debate', '').str.strip().str.replace('Transcripts', '').str.replace('Transcript', '').str.strip().str.replace('First Half', '').str.strip().str.replace('Second Half', '').str.strip()

df['Key'] = df['Debate_date'] + ' | ' + df['Speaker_standardized'].str.strip() # create the key for our mapping join

df = df.join(clean_map.set_index('Key'), rsuffix='_map', on = 'Key') # join to our mapping table

df.drop(['Speaker_fin', '0'], axis = 1, inplace = True) # drop some fields we don't need

df['Transcript'] = df['Transcript'].str.upper() # capitalize the transcript text for easier analysis

df.to_csv('Transcripts_full.csv') # send the full file to .csv

## We now have a clean set of transcripts. To help with various analyses, we created different cuts of the data

## The following code doesn't need to run -- only run if you want .csv files by year and year & speaker

## I also have this in the sentiment analysis to create these files for CANDIDATES ONLY

In [7]:
# create yearly files and individual candidate files

year_list = df['Year'].unique() # create unique list of years to loop through during file creation
speaker_list = df['Actual Speaker'].unique() # do the same for the speaker list

abs_dir = os.getcwd() # get current working directory

rel_dir = os.path.join(abs_dir, '\Yearly_Files\\') # set up path (I am on a Windows PC)

for i in year_list: # create a for loop
    if math.isnan(i) == False: # Make sure the iterator didn't pick up an empty year
        df_year = df[df['Year'] == i] # filter df for year
        df_year.to_csv(abs_dir + '\Yearly_Files\Transcripts_' + str(int(i)) + '.csv') # create year file
        for j in speaker_list: # do the same for the speakers...
            df_y_spk = df_year[df_year['Actual Speaker'] == j] # filter the year file for the speaker
            if len(df_y_spk.index) > 0: # make sure there are data in the df (just in case)
                
                # create year and speaker file
                df_y_spk.to_csv(abs_dir + '\Yearly_Files\Transcripts_' + j + '_' + str(int(i)) + '.csv') 
            else:
                pass
    else:
        pass
 